# vTTS - GPT-SoVITS v3 음성 클로닝 테스트

**Zero-shot Voice Cloning** - 참조 오디오 하나로 음성 클로닝!

## 🎯 GPT-SoVITS의 핵심

- **Zero-shot**: 5~10초 참조 오디오만으로 음성 클로닝
- **다국어**: 중국어, 영어, 일본어, 한국어, 광동어 지원
- **고품질**: v3/v4 최신 모델로 자연스러운 음성

⚠️ **중요**: GPT-SoVITS는 `reference_audio`와 `reference_text`가 **필수**입니다!

이 노트북에서는:
- ✅ **Part 0**: GPT-SoVITS 설치 및 환경 설정
- ✅ **Part 1**: 참조 오디오 준비
- ✅ **Part 2**: 음성 클로닝 테스트
- ✅ **Part 3**: Python 클라이언트 사용

## 🚀 Part 0: 설치 및 환경 설정

In [ ]:
# GPU 확인
import torch

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"CUDA Version: {torch.version.cuda}")
    DEVICE = "cuda"
else:
    print("⚠️ GPU 없음 - GPT-SoVITS는 GPU 권장!")
    DEVICE = "cpu"

In [ ]:
# GPT-SoVITS 설치 (시간이 오래 걸립니다 ~10-15분)
import os
import subprocess
import sys

print("🔧 GPT-SoVITS 환경 설정 중...")
print("   ⚠️ 이 과정은 10-15분 정도 소요됩니다.\n")

# 1. numpy 호환성
print("   [1/5] numpy 호환성 처리...")
!pip uninstall numpy -y -q
!pip install -q "numpy>=1.24.0,<2.0.0"

# 2. vTTS 기본 설치
print("   [2/5] vTTS 기본 설치...")
!pip install -q git+https://github.com/bellkjtt/vTTS.git

# 3. GPT-SoVITS 저장소 클론
print("   [3/5] GPT-SoVITS 저장소 클론...")
GPT_SOVITS_PATH = os.path.expanduser("~/.vtts/GPT-SoVITS")
os.makedirs(os.path.dirname(GPT_SOVITS_PATH), exist_ok=True)

if not os.path.exists(GPT_SOVITS_PATH):
    !git clone --depth 1 https://github.com/RVC-Boss/GPT-SoVITS.git {GPT_SOVITS_PATH}
else:
    print(f"   GPT-SoVITS already exists: {GPT_SOVITS_PATH}")

# 4. GPT-SoVITS requirements 설치
print("   [4/5] GPT-SoVITS 의존성 설치 (오래 걸림)...")
!pip install -q -r {GPT_SOVITS_PATH}/requirements.txt

# 5. 환경변수 설정
print("   [5/5] 환경변수 설정...")
os.environ["GPT_SOVITS_PATH"] = GPT_SOVITS_PATH

print(f"\n✅ GPT-SoVITS 설치 완료!")
print(f"   경로: {GPT_SOVITS_PATH}")

## 📁 Part 1: 참조 오디오 준비

GPT-SoVITS는 **참조 오디오**가 필수입니다!

- 권장 길이: 3~10초
- 깨끗한 음성 (배경 소음 없음)
- 참조 텍스트: 참조 오디오에서 말하는 내용

In [ ]:
# 참조 오디오 생성 (Supertonic 사용)
import os
import soundfile as sf

REF_DIR = "./reference_audio"
os.makedirs(REF_DIR, exist_ok=True)

print("🎙️ 참조 오디오 생성 중 (Supertonic 사용)...\n")

try:
    from vtts.engines.supertonic import SupertonicEngine
    from vtts.engines.base import TTSRequest
    
    # Supertonic 엔진 로드
    engine = SupertonicEngine(model_id="Supertone/supertonic-2", device=DEVICE)
    engine.load_model()
    
    # 참조 텍스트 (중요: 이 텍스트가 참조 오디오의 내용!)
    REF_TEXT_KO = "안녕하세요, 저는 음성 클로닝을 위한 참조 음성입니다."
    REF_TEXT_EN = "Hello, this is a reference audio for voice cloning."
    
    # 한국어 참조 오디오 생성
    request_ko = TTSRequest(
        text=REF_TEXT_KO,
        language="ko",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10}
    )
    output_ko = engine.synthesize(request_ko)
    
    REF_AUDIO_KO = os.path.join(REF_DIR, "reference_ko.wav")
    sf.write(REF_AUDIO_KO, output_ko.audio, output_ko.sample_rate)
    print(f"✅ 한국어 참조 오디오: {REF_AUDIO_KO}")
    print(f"   내용: {REF_TEXT_KO}")
    
    # 영어 참조 오디오 생성
    request_en = TTSRequest(
        text=REF_TEXT_EN,
        language="en",
        voice="F1",
        speed=1.0,
        extra_params={"total_steps": 10}
    )
    output_en = engine.synthesize(request_en)
    
    REF_AUDIO_EN = os.path.join(REF_DIR, "reference_en.wav")
    sf.write(REF_AUDIO_EN, output_en.audio, output_en.sample_rate)
    print(f"\n✅ 영어 참조 오디오: {REF_AUDIO_EN}")
    print(f"   내용: {REF_TEXT_EN}")
    
    engine.unload_model()
    print("\n✅ 참조 오디오 준비 완료!")
    
except Exception as e:
    print(f"⚠️ 참조 오디오 생성 실패: {e}")
    print("   직접 오디오 파일을 업로드하세요!")
    REF_AUDIO_KO = None
    REF_AUDIO_EN = None

In [ ]:
# 참조 오디오 재생
import IPython.display as ipd

print("🎧 참조 오디오 확인\n")

if REF_AUDIO_KO and os.path.exists(REF_AUDIO_KO):
    print(f"한국어 참조 오디오: {REF_TEXT_KO}")
    display(ipd.Audio(REF_AUDIO_KO))
    
if REF_AUDIO_EN and os.path.exists(REF_AUDIO_EN):
    print(f"\n영어 참조 오디오: {REF_TEXT_EN}")
    display(ipd.Audio(REF_AUDIO_EN))

## 🎙️ Part 2: GPT-SoVITS 서버 시작 및 음성 클로닝

In [ ]:
# GPT-SoVITS 서버 시작 (백그라운드)
import subprocess
import time
import httpx

print("🚀 GPT-SoVITS 서버 시작 중...")
print("   ⚠️ 모델 다운로드에 시간이 걸릴 수 있습니다.\n")

# 서버 시작 (백그라운드)
server_process = subprocess.Popen(
    ["vtts", "serve", "lj1995/GPT-SoVITS", 
     "--host", "0.0.0.0", 
     "--port", "8000",
     "--device", DEVICE],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

# 서버 준비 대기 (최대 5분)
print("   서버 준비 대기 중... (최대 5분)")
server_ready = False

for i in range(60):  # 60 * 5초 = 5분
    try:
        response = httpx.get("http://localhost:8000/health", timeout=5.0)
        if response.status_code == 200:
            server_ready = True
            break
    except:
        pass
    
    # 진행 표시
    if i % 6 == 0:
        print(f"   {i*5}초 경과...")
    time.sleep(5)

if server_ready:
    print("\n✅ GPT-SoVITS 서버 준비 완료!")
    print("   API: http://localhost:8000")
else:
    print("\n❌ 서버 시작 실패 - 로그 확인 필요")

In [ ]:
# 음성 클로닝 테스트 (한국어)
import httpx
import IPython.display as ipd

print("🎙️ 음성 클로닝 테스트 (한국어)\n" + "="*60)

# 합성할 텍스트
CLONE_TEXT_KO = "안녕하세요, GPT-SoVITS로 음성을 클로닝했습니다. 어떤가요?"

payload = {
    "model": "lj1995/GPT-SoVITS",
    "input": CLONE_TEXT_KO,
    "voice": "reference",
    "language": "ko",
    "response_format": "wav",
    "reference_audio": REF_AUDIO_KO,
    "reference_text": REF_TEXT_KO
}

try:
    print(f"입력 텍스트: {CLONE_TEXT_KO}")
    print(f"참조 오디오: {REF_AUDIO_KO}")
    print("\n합성 중... (1-2분 소요)")
    
    response = httpx.post(
        "http://localhost:8000/v1/audio/speech",
        json=payload,
        timeout=180.0  # 오래 걸릴 수 있음
    )
    
    if response.status_code == 200:
        # 결과 저장 및 재생
        output_path = "cloned_ko.wav"
        with open(output_path, "wb") as f:
            f.write(response.content)
        
        print(f"\n✅ 클로닝 완료!")
        print(f"   파일: {output_path}")
        print(f"   크기: {len(response.content)} bytes")
        
        print("\n🔊 원본 참조 오디오:")
        display(ipd.Audio(REF_AUDIO_KO))
        
        print("\n🔊 클로닝된 음성:")
        display(ipd.Audio(output_path))
    else:
        print(f"❌ 오류: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"❌ 요청 실패: {e}")

In [ ]:
# 음성 클로닝 테스트 (영어)
print("🎙️ 음성 클로닝 테스트 (영어)\n" + "="*60)

CLONE_TEXT_EN = "This is a voice cloned using GPT-SoVITS. The quality is amazing!"

payload = {
    "model": "lj1995/GPT-SoVITS",
    "input": CLONE_TEXT_EN,
    "voice": "reference",
    "language": "en",
    "response_format": "wav",
    "reference_audio": REF_AUDIO_EN,
    "reference_text": REF_TEXT_EN
}

try:
    print(f"입력 텍스트: {CLONE_TEXT_EN}")
    print(f"참조 오디오: {REF_AUDIO_EN}")
    print("\n합성 중... (1-2분 소요)")
    
    response = httpx.post(
        "http://localhost:8000/v1/audio/speech",
        json=payload,
        timeout=180.0
    )
    
    if response.status_code == 200:
        output_path = "cloned_en.wav"
        with open(output_path, "wb") as f:
            f.write(response.content)
        
        print(f"\n✅ 클로닝 완료!")
        print(f"   파일: {output_path}")
        
        print("\n🔊 원본 참조 오디오:")
        display(ipd.Audio(REF_AUDIO_EN))
        
        print("\n🔊 클로닝된 음성:")
        display(ipd.Audio(output_path))
    else:
        print(f"❌ 오류: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"❌ 요청 실패: {e}")

## 🐍 Part 3: Python 클라이언트 사용

In [ ]:
# vTTS Python 클라이언트 사용
from vtts import VTTSClient
import IPython.display as ipd

print("🐍 vTTS 클라이언트 사용법\n" + "="*60)

client = VTTSClient("http://localhost:8000")

# 음성 클로닝
try:
    audio = client.tts(
        text="파이썬 클라이언트로 음성 클로닝을 테스트합니다.",
        model="lj1995/GPT-SoVITS",
        voice="reference",
        language="ko",
        reference_audio=REF_AUDIO_KO,
        reference_text=REF_TEXT_KO
    )
    
    print(f"✅ 합성 완료!")
    print(f"   Duration: {len(audio.audio)/audio.sample_rate:.2f}s")
    print(f"   Sample Rate: {audio.sample_rate}Hz")
    
    # 저장 및 재생
    audio.save("client_cloned.wav")
    display(ipd.Audio(audio.audio, rate=audio.sample_rate))
    
except Exception as e:
    print(f"❌ 오류: {e}")

## 🧹 Part 4: 정리

In [ ]:
# 서버 종료
print("🛑 서버 종료 중...")

try:
    server_process.terminate()
    server_process.wait(timeout=5)
    print("✅ 서버 종료 완료")
except:
    server_process.kill()
    print("⚠️ 서버 강제 종료")

## 📝 요약

### GPT-SoVITS 사용 포인트

1. **참조 오디오 필수** - 클로닝할 음성의 샘플 필요 (3~10초)
2. **참조 텍스트 필수** - 참조 오디오에서 말하는 내용
3. **지원 언어** - zh, en, ja, ko, yue
4. **GPU 권장** - CPU도 가능하지만 매우 느림

### API 사용 예시

```python
from vtts import VTTSClient

client = VTTSClient("http://localhost:8000")

audio = client.tts(
    text="합성할 텍스트",
    model="lj1995/GPT-SoVITS",
    language="ko",
    reference_audio="./reference.wav",  # 필수!
    reference_text="참조 오디오의 내용"   # 필수!
)
audio.save("output.wav")
```

### 더 알아보기

- [vTTS GitHub](https://github.com/bellkjtt/vTTS)
- [GPT-SoVITS GitHub](https://github.com/RVC-Boss/GPT-SoVITS)
- [Docker 가이드](https://github.com/bellkjtt/vTTS/blob/main/DOCKER.md)